In [1]:
import pandas as pd
import numpy as np
from math import sqrt

In [2]:
df=pd.read_csv('resultado_consolidado.csv')
n_perguntas = len(df)/5
n = len(df)/30
print(n_perguntas, n)

1950.0 325.0


In [3]:
df.loc[df['resposta']=='@chimekarlla gosto muito de fazenda  , mas morro de medo de galinha kkkkkkk','gabarito']=0
df.loc[df['resposta']=='@behin_d_curtain Para mim, é precisamente o contrário  Vem a chuva e vem a boa disposição','gabarito']=1

In [4]:
perguntas = np.zeros((int(n), int(6)))
df["score"]= (df["gabarito"]==df["rotulo"])
df["score"] = df["score"].astype(int)
df['aceita']=0
print(perguntas)
df

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


,usuario,pergunta,resposta,tipo,gabarito,rotulo,score,aceita
0,0,0,Descobrir que tu tem q tomar remédio p vida in...,2,0,0,1,0
1,0,0,"aaaaaaaa amei tanto essas polaroids, nem sei e...",1,1,1,1,0
2,0,0,"@chimekarlla gosto muito de fazenda , mas mor...",0,0,1,0,0
3,0,0,"@behin_d_curtain Para mim, é precisamente o co...",0,1,1,1,0
4,0,0,tudo que eu não queria era estar em casa sozin...,0,0,0,1,0
...,...,...,...,...,...,...,...,...
9745,324,5,@anadantassa_ Feliz dia das crianças :),1,1,1,1,0
9746,324,5,eu cortei o cabeloooo e mds eu gostei mt,1,1,1,1,0
9747,324,5,Amanha vou ver venom :D,0,1,1,1,0
9748,324,5,Porque simplesmente as coisas que me deixam ch...,2,0,0,1,0


In [5]:
grouped = df.groupby(['usuario', 'pergunta', 'tipo'])
for name, group in grouped:
    if (name[2] == 1 or name[2]== 2):
        if (group['score'].sum()>=1):
            perguntas[name[0],name[1]] += 0.5
            if (perguntas[name[0],name[1]]==1):
                df.loc[(df['usuario'] ==name[0]) & (df['pergunta'] == name[1]), 'aceita'] = 1
                
        #print (name)
        #print (group)


In [6]:
a=df[df['tipo']==0].groupby(['pergunta']).agg({'score':'count', 'aceita': 'sum'}).reset_index().rename(columns={'pergunta':"estrategia",'score':'Sentenças a rotular'})

In [ ]:
def de_para(x):
    if (x==0):
        return ""

In [7]:
#quantidade de rotulações aceitas por estratégia
a['% Aceitação']=a['aceita']*100/a['Sentenças a rotular']
#a['estrat']=a['estrategia'].apply(lambda x: )
#a["Strategy"] = pd.DataFrame(['1_1',,,,]]
a=a.assign(strategy=pd.DataFrame(["1_1","1_2","1_3","2_1","2_2","3_1"]))
a

,estrategia,Sentenças a rotular,aceita,% Aceitação,strategy
0,0,975,921,94.461538,1_1
1,1,650,606,93.230769,1_2
2,2,325,202,62.153846,1_3
3,3,650,576,88.615385,2_1
4,4,325,316,97.230769,2_2
5,5,325,317,97.538462,3_1


In [8]:
#df.groupby(['pergunta']).filter(lambda x: (x['tipo'] == 0).any()).agg({'score':'sum', 'rotulo': 'count'}).rename(columns={'score':'Total sentenças'})
#acurácia das rotulações por estratégia
b=df[(df['tipo']==0) & (df['aceita']==1)].groupby(['pergunta']).agg({'rotulo': 'count','score':'sum'}).reset_index().rename(columns={'pergunta':"estrategia",'rotulo':'Total Aceitas','score':'Acertos'})
b['Acurácia (%)'] = b['Acertos']*100/b['Total Aceitas']
b=b.assign(strategy=pd.DataFrame(["1_1","1_2","1_3","2_1","2_2","3_1"]))
b

,estrategia,Total Aceitas,Acertos,Acurácia (%),strategy
0,0,921,721,78.284473,1_1
1,1,606,491,81.023102,1_2
2,2,202,164,81.188119,1_3
3,3,576,354,61.458333,2_1
4,4,316,219,69.303797,2_2
5,5,317,220,69.400631,3_1


In [9]:
aux1=df[df['tipo']==0].groupby(['resposta']).agg({'tipo': 'count','rotulo': 'mean','score':'sum', 'gabarito':'mean'}).reset_index().rename(columns={'tipo':'Total Respostas','rotulo':'Média','score':'Acertos'})
aux1['erro']=abs(aux1['gabarito']-aux1['Média'])
aux1# Média sem filtrar

,resposta,Total Respostas,Média,Acertos,gabarito,erro
0,"@Goncla_Silva és um gajo bacano, vê lá se não ...",325,0.575385,187,1,0.424615
1,@KingJokerLeto mas amiga eu to aqui ainda,325,0.246154,245,0,0.246154
2,"@RivasJairo Bravo, Jairo!!!",325,0.689231,224,1,0.310769
3,"@behin_d_curtain Para mim, é precisamente o co...",325,0.673846,219,1,0.326154
4,"@chimekarlla gosto muito de fazenda , mas mor...",325,0.335385,216,0,0.335385
5,Amanha vou ver venom :D,325,0.689231,224,1,0.310769
6,Já com saudades do meu nenê,325,0.387692,199,0,0.387692
7,Valoriza o coração do menininho que vc tem. El...,325,0.827692,269,1,0.172308
8,tudo que eu não queria era estar em casa sozin...,325,0.015385,320,0,0.015385
9,é tão lindo que dói,325,0.809231,263,1,0.190769


In [10]:
aux=df[(df['tipo']==0) & (df['aceita']==1)].groupby(['resposta']).agg({'tipo': 'count','rotulo': 'mean','score':'sum', 'gabarito':'mean'}).reset_index().rename(columns={'resposta':'Sentença','tipo':'Total Respostas','rotulo':'Média','score':'Acertos'})
aux# Filtro

,Sentença,Total Respostas,Média,Acertos,gabarito
0,"@Goncla_Silva és um gajo bacano, vê lá se não ...",288,0.590278,170,1
1,@KingJokerLeto mas amiga eu to aqui ainda,303,0.234323,232,0
2,"@RivasJairo Bravo, Jairo!!!",316,0.693038,219,1
3,"@behin_d_curtain Para mim, é precisamente o co...",307,0.687296,211,1
4,"@chimekarlla gosto muito de fazenda , mas mor...",307,0.332248,205,0
5,Amanha vou ver venom :D,317,0.694006,220,1
6,Já com saudades do meu nenê,288,0.361111,184,0
7,Valoriza o coração do menininho que vc tem. El...,303,0.854785,259,1
8,tudo que eu não queria era estar em casa sozin...,307,0.006515,305,0
9,é tão lindo que dói,202,0.811881,164,1


In [11]:
aux['u <=']=aux['Média']+1.96*aux['Média']*(1-aux['Média'])/ aux['Total Respostas'].apply(lambda x: sqrt(x))
aux['u >=']=aux['Média']-1.96*aux['Média']*(1-aux['Média'])/ aux['Total Respostas'].apply(lambda x: sqrt(x))


In [12]:
#estimador do u com 95% de confiança
aux

,Sentença,Total Respostas,Média,Acertos,gabarito,u <=,u >=
0,"@Goncla_Silva és um gajo bacano, vê lá se não ...",288,0.590278,170,1,0.618210,0.562346
1,@KingJokerLeto mas amiga eu to aqui ainda,303,0.234323,232,0,0.254526,0.214121
2,"@RivasJairo Bravo, Jairo!!!",316,0.693038,219,1,0.716494,0.669582
3,"@behin_d_curtain Para mim, é precisamente o co...",307,0.687296,211,1,0.711338,0.663255
4,"@chimekarlla gosto muito de fazenda , mas mor...",307,0.332248,205,0,0.357065,0.307430
5,Amanha vou ver venom :D,317,0.694006,220,1,0.717384,0.670629
6,Já com saudades do meu nenê,288,0.361111,184,0,0.387757,0.334465
7,Valoriza o coração do menininho que vc tem. El...,303,0.854785,259,1,0.868762,0.840809
8,tudo que eu não queria era estar em casa sozin...,307,0.006515,305,0,0.007239,0.005791
9,é tão lindo que dói,202,0.811881,164,1,0.832943,0.790819
